In [1]:
def idbi_bank(file_name):
    import tabula
    from tabula import read_pdf
    import pandas as pd
    import numpy as np
    import re
    import operator


    dt = pd.read_csv('IFSC codes1.csv', header=0)
    
    def find_ifsc4(branch_details, acc_no4):
        ifsc_obt = ""
        if branch_details != "":
            t = branch_details.replace("-", ",").split(",")        # t[0]
            ifsc_found = dt.loc[operator.and_(operator.and_(dt["Name of the bank"] == "IDBI" , dt["Branch Name"].str.contains("{}".format(t[0].strip(" ")))),dt["Address"].str.contains("{}".format(t[-1].strip(" "))))]
            if "IBKL" in str(ifsc_found["IFSC Code"]):
                index = str(ifsc_found).find("IBKL")
                ifsc_obt = str(ifsc_found)[index: index+11]
            elif "Empty" in str(ifsc_found):
                dts = dt.loc[dt["IFSC Code"] == "IBKL000{}".format((acc_no4).strip(" ").strip(":").strip(" ")[:4])]
                if len(dts)== 1:
                    ifsc_obt = ("IBKL000"+"{}".format(acc_no4.strip(" ").strip(":").strip(" ")[:4]))
                else:
                    print("Branch not found in the dataset!!")
        else:
            dts = dt.loc[dt["IFSC Code"] == "IBKL000{}".format((acc_no4).strip(" ").strip(":").strip(" ")[:4])]
            if len(dts)== 1:
                ifsc_obt = ("IBKL000"+"{}".format((acc_no4).strip(" ").strip(":").strip(" ")[:4]))
            else:
                print("Branch not found in the dataset!!")

        return ifsc_obt
    
    
    def type_2():
        acc_n4 = ""; acc_n4s = ""; acc_no4 = ""; addr4 = ""; br_details4 = "";
        cifn4 = ""; ifsc4 = ""; micrn4 = "";
        line = 0
        siz = len(df4.columns)

        for count,i in enumerate(df4[df4.columns[0]]):
            try:
                if i.isupper():
                    line += 1
                    if line == 1:
                        acc_n4 = i
                    elif line == 2:
                        addr4 = (df4[df4.columns[0]][count]).strip(" ").strip(".").strip(":").strip(" ") + " " + (df4[df4.columns[0]][count+1]).strip(" ").strip(".").strip(":").strip(" ") + " " + (df4[df4.columns[0]][count+2]).strip(" ").strip(".").strip(":").strip(" ")
                if "account number" in i.lower():
                    if (i.lower().strip("account number").strip(" ").strip(".").strip(":").strip(" ")) != "":
                        acc_no4 = (i.lower().strip("account number").strip(" ").strip(".").strip(":").strip(" "))
                    else:
                        acc_no4 = (df4[df4.columns[0]][count+1]).strip(" ").strip(".").strip(":").strip(" ")
            except AttributeError:
                pass
        return acc_n4, acc_no4, addr4
    
    
    tables=read_pdf(file_name, area=(15, 15, 260, 800), pages = 1)
    df4 = pd.DataFrame()
    df4 = pd.concat([c for c in tables]).drop_duplicates()
    df4.reset_index(drop=True, inplace=True)
    df4.dropna(axis=1, how="all", thresh=None, subset=None, inplace=True)
    df4 = pd.DataFrame(np.row_stack([df4.columns, df4.values]),columns=['1', '2', '3'])
    
    acc_n4 = ""; acc_n4s = ""; acc_no4 = ""; addr4 = ""; br_details4 = "";
    cifn4 = ""; ifsc4 = ""; micrn4 = "";
    line = 0
    siz = len(df4.columns)

    for count,i in enumerate(df4[df4.columns[0]]):
        try:
            if "primary" in i.lower():
                if (i.lower().strip("primary").strip(" ").strip(".").strip(":").strip(" ")) != "":
                    acc_n4 = (i.lower().strip("primary").strip(" ").strip(".").strip(":").strip(" "))
                else:
                    acc_n4 = (df4[df4.columns[1]][count]).strip(" ").strip(".").strip(":").strip(" ")
            if "secondary" in i.lower():
                if (i.lower().strip("secondary").strip(" ").strip(".").strip(":").strip(" ")) != "":
                    acc_n4s = (i.lower().strip("secondary").strip(" ").strip(".").strip(":").strip(" "))
                else:
                    acc_n4s = (df4[df4.columns[1]][count]).strip(" ").strip(".").strip(":").strip(" ")
            if "account no" in i.lower():
                if (i.lower().strip("account no").strip(" ").strip(".").strip(":").strip(" ")) != "":
                    acc_no4 = (i.lower().strip("account no").strip(" ").strip(".").strip(":").strip(" "))
                else:
                    acc_no4 = (df4[df4.columns[1]][count]).strip(" ").strip(".").strip(":").strip(" ")
            if "account branch" in i.lower():
                if (i.lower().strip("account branch").strip(" ").strip(".").strip(":").strip(" ")) != "":
                    br_details4 = (i.lower().strip("account branch").strip(" ").strip(".").strip(":").strip(" "))
                else:
                    br_details4 = (df4[df4.columns[1]][count]).strip(" ").strip(".").strip(":").strip(" ")
            if "address" in i.lower():
                if (i.lower().strip("address").strip(" ").strip(".").strip(":").strip(" ")) != "":
                    addr4 = (i.lower().strip("address").strip(" ").strip(".").strip(":").strip(" "))
                else:
                    addr4 = (df4[df4.columns[1]][count]).strip(" ").strip(".").strip(":").strip(" ") + " " + (df4[df4.columns[1]][count+1]).strip(" ").strip(".").strip(":").strip(" ") + " " + (df4[df4.columns[1]][count+2]).strip(" ").strip(".").strip(":").strip(" ")

        except AttributeError:
            pass


    if acc_n4 != "":
        dfr4 = pd.DataFrame([["Account Name", acc_n4]], columns = ["Customer Information", "Data_Extracted"])
    else:
        acc_n4, acc_no4, addr4 = type_2()
        dfr4 = pd.DataFrame([["Account Name", acc_n4]], columns = ["Customer Information", "Data_Extracted"])
    dfr4.loc[len(dfr4)] = ["Account Number", acc_no4]
    if ifsc4 != "":
        dfr4.loc[len(dfr4)] = ["IFS Code", ifsc4]
    else:
        dfr4.loc[len(dfr4)] = ["IFS Code", find_ifsc4(br_details4, acc_no4)]
    dfr4.loc[len(dfr4)] = ["Branch Name", br_details4]
    dfr4.loc[len(dfr4)] = ["Address", addr4]
    dfr4.loc[len(dfr4)] = ["MICR Code", micrn4]
    dfr4.loc[len(dfr4)] = ["CIF Number", cifn4]

    return dfr4

In [3]:
idbi_bank("IDBI2.pdf")

Got stderr: Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Mangal,Bold are not implemented in PDFBox and will be ignored
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Apr 03, 2020 3:25:20 PM org.ap

,Customer Information,Data_Extracted
0,Account Name,GARAM MASALA COLLECTION
1,Account Number,0571102000015233
2,IFS Code,IBKL0000571
3,Branch Name,
4,Address,F 22 MOKSH PLAZA S V ROAD BORIVALI WEST MUMBAI...
5,MICR Code,
6,CIF Number,
